In [ ]:
print(__doc__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy
import os
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import  VotingClassifier
from sklearn.externals import joblib

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

def ROCPlt(fpr,tpr,roc_auc,n_classes):
    plt.figure()
    lw = 1
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()
    
def ReportWithROC(name,model,X,Y):
    X = X
    y = Y
    print(y)
    
    # Add noisy features to make the problem harder
    random_state = np.random.RandomState(0)
    n_samples, n_features = X.shape
    print(X.shape)
    X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]


    # shuffle and split training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1,random_state=0)
    
    
    Report(name,model,X, y, X_train, y_train,X_test,y_test)
    
    y_train=label_binarize(y_train, classes=[0, 1, 2])
    y_test=label_binarize(y_test, classes=[0, 1, 2])
    print(y_train.dtype)
    print(y)
    n_classes =y_train.shape[1]-1
    print(y_train.shape[1])
    print(n_classes)

    # Learn to predict each class against the other
    classifier = OneVsRestClassifier(model)
    y_score = classifier.fit(X_train, y_train).predict(X_test)
 
       
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
    
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    #return fpr,tpr,roc_auc,n_classes
    ROCPlt(fpr,tpr,roc_auc,n_classes)
    return fpr,tpr,roc_auc,n_classes

def Report(name,model,X, Y, X_train,y_train,X_test,y_test):
    print(name)
    print('cross_val_score')
    kfold = KFold(n_splits=10, shuffle=True, random_state=0)
    results = cross_val_score(model, X, Y, cv=kfold)
    print("cross_val_score: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print('accuracy score: {}'.format(accuracy))
    #Accuracy of the predicted values
    print('confusion_matrix')
    print(classification_report(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    
    #save model 
    joblib.dump(model, name+'.sav')
    return ;

seed = 10

    # Define Dataset
dataset = pd.read_excel('LD.xlsx')
X = dataset.iloc[:,0:3].values
Y = dataset.iloc[:,3].values

fpr= [None] * 10
tpr=[None] * 10
roc_auc=[None] * 10
n_classes=[None] * 10


# Implementing Classifiers 

model1 = DecisionTreeClassifier(criterion='gini',min_samples_leaf=5,min_samples_split=5,max_depth=None,random_state=seed)
fpr[0],tpr[0],roc_auc[0],n_classes[0]=ReportWithROC('DecisionTreeClassifier',model1,X,Y)
print('fpr:',fpr[0])

model2 = svm.SVC(gamma='auto')
fpr[1],tpr[1],roc_auc[1],n_classes[1]=ReportWithROC('svm.SVC',model2,X,Y)

model3 = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
fpr[2],tpr[2],roc_auc[2],n_classes[2]=ReportWithROC('LogisticRegression',model3,X,Y)


model4 = GaussianNB()
fpr[3],tpr[3],roc_auc[3],n_classes[3]=ReportWithROC('GaussianNB',model4,X,Y)

model5 = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0,random_state=None)
fpr[4],tpr[4],roc_auc[4],n_classes[4]=ReportWithROC('AdaBoostClassifier',model5,X,Y)

model6=RandomForestClassifier(n_estimators=10, max_depth=2,random_state=0)
fpr[5],tpr[5],roc_auc[5],n_classes[5]=ReportWithROC('RandomForestClassifier',model6,X,Y)

model7=KNeighborsClassifier(n_neighbors=3)
fpr[6],tpr[6],roc_auc[6],n_classes[6]=ReportWithROC('KNeighborsClassifier',model7,X,Y)

model8=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
fpr[7],tpr[7],roc_auc[7],n_classes[7]=ReportWithROC('MLPClassifier',model8,X,Y)

model9 = GradientBoostingClassifier(n_estimators=20, max_features=2, max_depth = 2, random_state = 0)
fpr[8],tpr[8],roc_auc[8],n_classes[8]=ReportWithROC('GradiantBoostClassifier',model9,X,Y)


model10=VotingClassifier(estimators=[
    (' model1', model1)
   ,(' model2', model2)
    ,(' model3', model3)
    ,(' model4', model4)
    ,(' model5', model5)
    ,(' model6', model6)
    ,(' model7', model7)
    ,(' model8', model8)
    ,(' model9', model9)
     ]
     , voting='hard'
    #,voting='soft'
                                       
                       )

fpr[9],tpr[9],roc_auc[9],n_classes[9]=ReportWithROC('VotingClassifier',model10,X,Y)



# Aggregating ROC Curves
#p="macro"
#p=0
p=1
#p='micro'
plt.figure(figsize=(15, 10))
colors = cycle(['brown', 'darkorange', 'cornflowerblue','black','green','yellow','c','y','purple','red'])
for i, color in zip(range(10), colors):
    if fpr[i]!=None:
        plt.plot(fpr[i][p], tpr[i][p]
         ,label='model '+str(i+1) +' ROC curve for '+str(p)+' (area = {0:0.2f})'
                   ''.format(roc_auc[i][p]),
             color=color, linestyle=':', linewidth=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()




